In [ ]:
import os
import csv
import requests
import base64
from IPython.display import display, FileLink

# 配置参数
image_dir = "/home/luany/桌面/moneycount/202507"  # 替换为你的图片目录路径
image_prefixes = ["jh_", "zs_"]  # 多个图片名称前缀
output_csv = "combined_ocr_results.csv"  # 输出CSV文件名
custom_prompt = """
总结下面每一条的文字和金额，忽略卡号（上传的图片）
有下面数据，按下面几个列名称 id, pay_time, pay_monthyear, pay_source, pay_note, pay_money, pay_tag, app_source 按规则组成csv：
<规则>
id 为 空，
pay_time 为 2025-01-01 00:00:01 并且每一条依次加一秒，
pay_monthyear 为 01_25，
pay_source 为 依据文件名划分，如文件名前缀jh_是<建设银行></建设银行>，如文件名前缀zs_是<招商银行></招商银行>
pay_note 为 数据中的文字描述
pay_money 为金额，此项为正数
pay_tag  为 credit，
app_source 为 BK
</规则>
<数据>

<建设银行>
商户分期2/3: 蚂蚁宝 - 46.33元
分期1/12: 账单分期单期金额 - 142.11元
账单分期每期利息 - 10.23元
分期3/3: 账单分期单期金额 - 1,252.67元
</建设银行>

<招商银行>
账单分期每期利息 - 28.18元
</招商银行>

</数据>
"""

# DeepSeek API配置（根据实际API调整）
DEEPSEEK_API_URL = "https://api.deepseek.com/v1/ocr"
API_KEY = "sk-"  # 替换为你的实际API密钥
MAX_RETRIES = 3  # 最大重试次数

def encode_image_to_base64(image_path):
    """将图片编码为base64"""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def call_deepseek_ocr(image_path, prompt):
    """调用DeepSeek OCR API（带重试机制）"""
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }
    
    payload = {
        "image": encode_image_to_base64(image_path),
        "prompt": prompt,
        "response_format": "csv"
    }
    
    for attempt in range(MAX_RETRIES):
        try:
            response = requests.post(DEEPSEEK_API_URL, json=payload, headers=headers, timeout=30)
            response.raise_for_status()
            return response.text.strip()
        except requests.exceptions.RequestException as e:
            if attempt == MAX_RETRIES - 1:
                print(f"图片 {os.path.basename(image_path)} 处理失败，已达最大重试次数: {str(e)}")
                return None
            print(f"图片 {os.path.basename(image_path)} 第 {attempt+1} 次尝试失败，正在重试...")

def find_matching_images(directory, prefixes):
    """查找目录中匹配任一前缀的图片文件"""
    matched_files = []
    valid_extensions = ('.png', '.jpg', '.jpeg', '.bmp', '.webp')
    
    for filename in os.listdir(directory):
        lower_filename = filename.lower()
        if any(lower_filename.startswith(prefix.lower()) for prefix in prefixes) and lower_filename.endswith(valid_extensions):
            matched_files.append(filename)
    
    return sorted(matched_files)

def process_multiple_prefixes(image_dir, prefixes, prompt, output_file):
    """处理多个前缀的图片并生成合并的CSV文件"""
    # 查找所有匹配的图片文件
    image_files = find_matching_images(image_dir, prefixes)
    
    if not image_files:
        print(f"在目录 {image_dir} 中未找到以 {prefixes} 开头的图片文件")
        return
    
    print(f"找到 {len(image_files)} 个匹配的图片文件，开始处理...")
    
    all_results = []
    header = None
    
    for idx, img_file in enumerate(image_files, 1):
        img_path = os.path.join(image_dir, img_file)
        print(f"正在处理 {idx}/{len(image_files)}: {img_file}")
        
        # 调用OCR API
        result = call_deepseek_ocr(img_path, prompt)
        
        if not result:
            continue
            
        # 解析CSV结果并添加图片文件名
        lines = result.split('\n')
        current_header = lines[0].split(',')
        
        # 确保所有结果有相同的列结构
        if header is None:
            header = current_header
            if '图片文件名' not in header:
                header.append('图片文件名')
        
        for line in lines[1:]:
            if line.strip():
                row = line.split(',')
                # 确保行数据与标题长度匹配
                if len(row) == len(current_header):
                    if len(row) < len(header):
                        row.extend([''] * (len(header) - len(row) - 1))
                    row.append(img_file)  # 添加图片文件名
                    all_results.append(row)
    
    # 写入CSV文件
    if header and all_results:
        with open(output_file, 'w', newline='', encoding='utf-8-sig') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(header)
            writer.writerows(all_results)
        
        print(f"\n处理完成！共成功处理 {len(all_results)} 条数据")
        print(f"结果已保存到: {output_file}")
        
        # 在Jupyter中显示下载链接
        display(FileLink(output_file))
    else:
        print("没有有效数据可保存")

# 在Jupyter中执行处理
process_multiple_prefixes(image_dir, image_prefixes, custom_prompt, output_csv)

找到 2 个匹配的图片文件，开始处理...
正在处理 1/2: jh_20250802-163603.jpg
图片 jh_20250802-163603.jpg 第 1 次尝试失败，正在重试...
图片 jh_20250802-163603.jpg 第 2 次尝试失败，正在重试...
图片 jh_20250802-163603.jpg 处理失败，已达最大重试次数: 404 Client Error: Not Found for url: https://api.deepseek.com/v1/ocr
正在处理 2/2: zs_20250802-163615.jpg
图片 zs_20250802-163615.jpg 第 1 次尝试失败，正在重试...
图片 zs_20250802-163615.jpg 第 2 次尝试失败，正在重试...
图片 zs_20250802-163615.jpg 处理失败，已达最大重试次数: 404 Client Error: Not Found for url: https://api.deepseek.com/v1/ocr
没有有效数据可保存
